# Imports

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas
import os
from utils import get_cities_name

# Variables

In [3]:
link_list = pandas.read_csv('../data/clean_urls_list.csv')
urls = link_list['urls']
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
updated_urls_list = []

In [4]:
for url in urls : 
    new_url = url.replace("cost-of-living", "quality-of-life")
    updated_urls_list.append(new_url)

# Functions

In [5]:
def get_safety_index(urls_to_iterate) : 
    print('getting safety index .', end=' ')
    safety_index = []
    for url in urls_to_iterate : 
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        safety_td = soup.select_one(f"td:-soup-contains('Safety Index')")
        if safety_td is None : 
            safety_index.append(None)
            continue
        next_td = safety_td.find_next_sibling("td")
        safety_index.append(next_td.text)
        print('.', end=' ', flush=True)
    print()
    return safety_index


In [6]:
def get_safety_per_city(urls):
    cities = get_cities_name(urls)
    safety_index = get_safety_index(urls)
    
    if len(cities) != len(safety_index):
        print("Error, number of cities and safety index not equal.")
        return
    
    data = {'City': cities, 'Safety Index': safety_index}
    df = pandas.DataFrame(data)
    return df

# Data collection

In [7]:
safety_index_per_city = get_safety_per_city(updated_urls_list)

getting city names . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
getting safety index . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 


# Save Data

In [8]:
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
output_path = os.path.join(project_root, 'data', 'safety_index_by_city.csv')

safety_index_per_city.to_csv(output_path, index=False)